# Visualizing ALDEx2 feature differentials using Qurro

In this example, we use transcriptomic data from [TCGA](https://portal.gdc.cancer.gov/repository). We downloaded 
100 gene expression files from lung squamous cell carcinoma (LUSC) primary tumors and 49 solid tissue normal expression files. We have pre-processed this data into a feature table for ease of use, but the gdc manifest file is also provided for your convenience as well as the script used to aggregate all the files.

Please note that this is primarily an example to demonstrate how to use Qurro and should not be taken as an example of how best to use ALDEx2 or transcriptomic data.

[1] J. N. Weinstein et al., “The Cancer Genome Atlas Pan-Cancer analysis project,” *Nat Genet*, vol. 45, no. 10, pp. 1113–1120, Oct. 2013, doi: 10.1038/ng.2764.


## Requirements:

This notebook requires Qurro, pandas (< 1.0.0), and biom to be installed for Python. The R package [ALDEx2](http://bioconductor.org/packages/release/bioc/html/ALDEx2.html) is also required for the `run_aldex.R` script.

## 0. Setting up
In this section, we replace the output directory with an empty directory. This just lets us run this notebook multiple times, without any tools complaining about overwriting files.

In [1]:
# Clear the output directory so we can write these files there
!rm -rf output/*
# Since git doesn't keep track of empty directories, create the output/ directory if it doesn't already exist
# (if it does already exist, -p ensures that an error won't be thrown)
!mkdir -p output

## 1. Processing the feature table

The original feature table has over 60,000 features which is too computationally expensive to process for this example. We will filter this table to use the top 1000 genes by total abundance.

In [2]:
import os
import re
import subprocess
import pandas as pd

In [3]:
feature_table = pd.read_csv(
    "input/TCGA_LUSC_expression_feature_table.tsv",
    sep="\t",
    index_col=0,
)
print(feature_table.shape)
feature_table.head()

(60483, 149)


,TCGA-43-5670-11A,TCGA-77-8008-11A,TCGA-18-3410-01A,TCGA-43-6771-11A,TCGA-66-2758-01A,TCGA-90-7767-01A,TCGA-66-2795-01A,TCGA-77-7138-01A,TCGA-39-5019-01A,TCGA-90-6837-11A,...,TCGA-56-7823-01B,TCGA-77-7142-11A,TCGA-22-5483-11A,TCGA-77-8153-01A,TCGA-85-A4JC-01A,TCGA-39-5040-11A,TCGA-43-6143-11A,TCGA-77-7335-11A,TCGA-85-7698-01A,TCGA-43-2581-01A
feature-id,,,,,,,,,,,,,,,,,,,,,
ENSG00000000003.13,4707,1016,2286,1238,2982,2795,1516,5507,5504,2079,...,1988,1059,1297,1970,2450,1432,1500,1790,3722,2471
ENSG00000000005.5,5,1,1,5,1,0,0,0,2,3,...,1,5,4,0,0,4,3,0,1,0
ENSG00000000419.11,2019,1731,2123,2121,3868,2722,2130,4834,4557,1264,...,1194,1449,1382,747,1741,3167,1454,1569,2477,1826
ENSG00000000457.12,1062,968,1883,655,800,1987,935,898,1014,840,...,449,883,614,678,305,705,917,658,961,470
ENSG00000000460.15,204,202,1923,205,706,2191,956,927,1347,264,...,748,159,180,887,275,181,206,153,581,770


In [4]:
feature_table_filt = feature_table.loc[
    feature_table.sum(axis=1).sort_values(ascending=False).head(1000).index, :
]
print(feature_table_filt.shape)
feature_table_filt.head()

(1000, 149)


,TCGA-43-5670-11A,TCGA-77-8008-11A,TCGA-18-3410-01A,TCGA-43-6771-11A,TCGA-66-2758-01A,TCGA-90-7767-01A,TCGA-66-2795-01A,TCGA-77-7138-01A,TCGA-39-5019-01A,TCGA-90-6837-11A,...,TCGA-56-7823-01B,TCGA-77-7142-11A,TCGA-22-5483-11A,TCGA-77-8153-01A,TCGA-85-A4JC-01A,TCGA-39-5040-11A,TCGA-43-6143-11A,TCGA-77-7335-11A,TCGA-85-7698-01A,TCGA-43-2581-01A
feature-id,,,,,,,,,,,,,,,,,,,,,
ENSG00000168484.11,972916,843573,223,1850407,1139,135,488,2352,1482,1815346,...,386,846652,1635957,42,6,1734787,2453372,1001158,699,490
ENSG00000185303.14,975800,887399,10662,1711785,29460,458,3623,4214,178301,686350,...,501,860576,2331898,1074,8,1001212,1513129,967935,2460,7480
ENSG00000198804.2,550882,567646,391264,765533,222015,88019,119269,206227,522146,380501,...,141559,449164,439839,252347,252814,353950,881008,166618,243607,458559
ENSG00000122852.13,786296,663972,8201,1509224,15981,1544,2141,3157,151564,752807,...,432,663762,2079926,522,16,1166295,1096501,906560,780,7326
ENSG00000198886.2,315275,499467,179894,626067,194809,115463,104261,279515,396785,404277,...,262923,442133,429011,309030,218573,241436,696461,131715,252738,276778


We also want to strip everything after the period in the Ensemble IDS. For example, `ENSG00000000003.13` should be converted to `ENSG00000000003`.

In [5]:
feature_table_filt.index = [re.search("ENSG[0-9]*", x).group() for x in feature_table_filt.index]
feature_table_filt.index.name = "feature-id"
feature_table_filt.head()

,TCGA-43-5670-11A,TCGA-77-8008-11A,TCGA-18-3410-01A,TCGA-43-6771-11A,TCGA-66-2758-01A,TCGA-90-7767-01A,TCGA-66-2795-01A,TCGA-77-7138-01A,TCGA-39-5019-01A,TCGA-90-6837-11A,...,TCGA-56-7823-01B,TCGA-77-7142-11A,TCGA-22-5483-11A,TCGA-77-8153-01A,TCGA-85-A4JC-01A,TCGA-39-5040-11A,TCGA-43-6143-11A,TCGA-77-7335-11A,TCGA-85-7698-01A,TCGA-43-2581-01A
feature-id,,,,,,,,,,,,,,,,,,,,,
ENSG00000168484,972916,843573,223,1850407,1139,135,488,2352,1482,1815346,...,386,846652,1635957,42,6,1734787,2453372,1001158,699,490
ENSG00000185303,975800,887399,10662,1711785,29460,458,3623,4214,178301,686350,...,501,860576,2331898,1074,8,1001212,1513129,967935,2460,7480
ENSG00000198804,550882,567646,391264,765533,222015,88019,119269,206227,522146,380501,...,141559,449164,439839,252347,252814,353950,881008,166618,243607,458559
ENSG00000122852,786296,663972,8201,1509224,15981,1544,2141,3157,151564,752807,...,432,663762,2079926,522,16,1166295,1096501,906560,780,7326
ENSG00000198886,315275,499467,179894,626067,194809,115463,104261,279515,396785,404277,...,262923,442133,429011,309030,218573,241436,696461,131715,252738,276778


There are 149 samples but 135 cases - some of the patients have both their primary tumor and normal present in the feature table. To better facilitate coparisons, we're going to only keep the normal samples for those patients who have both. In the barcode the last 3 digits represent the sample type. `01A` or `01B` correspond to tumor sample, while `11A` corresponds to solid tissue normal.

In [6]:
from collections import Counter

all_cases = [re.search("TCGA-[A-Za-z0-9]{2}-[A-Za-z0-9]{4}", x).group() for x in feature_table_filt.columns]
duplicated_cases = [barcode for barcode, count in Counter(all_cases).items() if count > 1]
len(duplicated_cases)

14

In [7]:
samples_to_remove = []
for col in feature_table_filt:
    case, sample = re.search("(TCGA-[A-Za-z0-9]{2}-[A-Za-z0-9]{4})-([0-1]{2}[AB])", col).groups()
    if case in duplicated_cases and sample.startswith("01"):
        samples_to_remove.append(col)

Now that each patient only has one sample represented in the feature table, we can use case barcodes (TCGA-XX-YYYY) instead of sample barcodes (TCGA-XX-YYYY-ZZ). This will allow us to more easily compare across our metadata conditions.

In [8]:
feature_table_filt = feature_table_filt.drop(columns=samples_to_remove)
feature_table_filt.columns = [
    re.search("TCGA-[A-Za-z0-9]{2}-[A-Za-z0-9]{4}", x).group() for x in feature_table_filt.columns
]
print(feature_table_filt.shape)
feature_table_filt.head()

(1000, 135)


,TCGA-43-5670,TCGA-77-8008,TCGA-18-3410,TCGA-43-6771,TCGA-66-2758,TCGA-66-2795,TCGA-39-5019,TCGA-90-6837,TCGA-77-A5GB,TCGA-52-7810,...,TCGA-85-8582,TCGA-77-7142,TCGA-22-5483,TCGA-77-8153,TCGA-85-A4JC,TCGA-39-5040,TCGA-43-6143,TCGA-77-7335,TCGA-85-7698,TCGA-43-2581
feature-id,,,,,,,,,,,,,,,,,,,,,
ENSG00000168484,972916,843573,223,1850407,1139,488,1482,1815346,4,68,...,156,846652,1635957,42,6,1734787,2453372,1001158,699,490
ENSG00000185303,975800,887399,10662,1711785,29460,3623,178301,686350,451,1731,...,236,860576,2331898,1074,8,1001212,1513129,967935,2460,7480
ENSG00000198804,550882,567646,391264,765533,222015,119269,522146,380501,163642,321512,...,198960,449164,439839,252347,252814,353950,881008,166618,243607,458559
ENSG00000122852,786296,663972,8201,1509224,15981,2141,151564,752807,265,947,...,168,663762,2079926,522,16,1166295,1096501,906560,780,7326
ENSG00000198886,315275,499467,179894,626067,194809,104261,396785,404277,172102,414434,...,174529,442133,429011,309030,218573,241436,696461,131715,252738,276778


In [9]:
feature_table_filt.to_csv(
    "output/TCGA_LUSC_expression_feature_table_filt.tsv",
    sep="\t",
    index=True,
)

## 2. Process sample metadata

We want to include several fields in the sample metadata:

1. Sample type (tumor or normal)
2. Race
3. Age at diagnosis
4. Gender
5. Cigarettes per day
6. Years smoked

In [10]:
sample_sheet = pd.read_csv("input/gdc_sample_sheet.2020-02-13.tsv", sep="\t")
sample_sheet.head()

,File ID,File Name,Data Category,Data Type,Project ID,Case ID,Sample ID,Sample Type
0,e4c62f17-d1e8-4543-9b7e-daa2b68306e0,bc5be208-5934-40dd-81df-567599ea2a51.htseq.cou...,Transcriptome Profiling,Gene Expression Quantification,TCGA-LUSC,TCGA-33-6737,TCGA-33-6737-01A,Primary Tumor
1,220a03f7-7ab6-4233-8f65-7ac5decca4b9,60040f95-8414-4956-bd8a-ec461a49207c.htseq.cou...,Transcriptome Profiling,Gene Expression Quantification,TCGA-LUSC,TCGA-18-3410,TCGA-18-3410-01A,Primary Tumor
2,8894c42e-ce65-4088-88e3-921ce7165261,950e2ba0-a247-4bd6-8092-f97cc4018a79.htseq.cou...,Transcriptome Profiling,Gene Expression Quantification,TCGA-LUSC,TCGA-33-A4WN,TCGA-33-A4WN-01A,Primary Tumor
3,daa44ce1-1671-46b9-aa48-2f4155f0ee49,a998a5b1-397d-4497-a58c-9b9e1c7f491e.htseq.cou...,Transcriptome Profiling,Gene Expression Quantification,TCGA-LUSC,TCGA-56-7579,TCGA-56-7579-01A,Primary Tumor
4,ef056c34-c2b9-47dd-afbf-ed81fc16dc74,840bb854-0669-485e-9d83-c4e1e4f10626.htseq.cou...,Transcriptome Profiling,Gene Expression Quantification,TCGA-LUSC,TCGA-34-5236,TCGA-34-5236-01A,Primary Tumor


In [11]:
sample_sheet_new = sample_sheet.set_index("Case ID", drop=True)
sample_sheet_new = sample_sheet_new[["Sample Type"]]
sample_sheet_new.head()

,Sample Type
Case ID,
TCGA-33-6737,Primary Tumor
TCGA-18-3410,Primary Tumor
TCGA-33-A4WN,Primary Tumor
TCGA-56-7579,Primary Tumor
TCGA-34-5236,Primary Tumor


In [12]:
clinical = pd.read_csv(
    "input/clinical.tsv", 
    sep="\t", 
    na_values=["--", "not reported"],
    index_col="submitter_id"
)
clinical = clinical[["age_at_diagnosis", "race", "gender"]]
clinical.head()

,age_at_diagnosis,race,gender
submitter_id,,,
TCGA-43-5670,25652.0,white,male
TCGA-43-5670,25652.0,white,male
TCGA-66-2800,25810.0,NaN,male
TCGA-66-2800,25810.0,NaN,male
TCGA-66-2788,20637.0,NaN,male


In [13]:
exposure = pd.read_csv(
    "input/exposure.tsv",
    sep="\t",
    na_values=["--", "Not Reported"],
    index_col="submitter_id"
)
exposure = exposure[["cigarettes_per_day", "years_smoked"]]
exposure.head()

,cigarettes_per_day,years_smoked
submitter_id,,
TCGA-43-5670,1.643836,10.0
TCGA-66-2800,4.109589,50.0
TCGA-66-2788,4.383562,40.0
TCGA-77-7338,2.260274,NaN
TCGA-56-7222,1.260274,NaN


In [14]:
sample_sheet_clinical_exposure = sample_sheet_new.join(clinical).join(exposure)
sample_sheet_clinical_exposure.index.name = "Sample ID"
sample_sheet_clinical_exposure = sample_sheet_clinical_exposure.loc[
    ~sample_sheet_clinical_exposure.index.duplicated(keep="first"), :
]
print(sample_sheet_clinical_exposure.shape)
sample_sheet_clinical_exposure.head()

(135, 6)


,Sample Type,age_at_diagnosis,race,gender,cigarettes_per_day,years_smoked
Sample ID,,,,,,
TCGA-18-3410,Primary Tumor,29827.0,NaN,male,NaN,NaN
TCGA-18-3411,Primary Tumor,23370.0,NaN,female,2.739726,NaN
TCGA-18-3416,Primary Tumor,30435.0,NaN,male,2.191781,NaN
TCGA-18-4086,Primary Tumor,23731.0,NaN,male,1.643836,NaN
TCGA-18-5595,Primary Tumor,18611.0,NaN,male,NaN,NaN


In [15]:
sample_sheet_clinical_exposure.to_csv("output/sample_metadata.tsv", sep="\t", index=True)

## 3. Running ALDEx2

ALDEx2 is a tool used for differential abundance between conditions. For the sake of demonstration, we will compare the tumor samples to the normal samples. Your own application may be more complicated if there are multiple differential abundance conditions. We have provided a script to run ALDEx2 on the processed feature table. Note that this step can take several minutes.

[2] A. D. Fernandes, J. M. Macklaim, T. G. Linn, G. Reid, and G. B. Gloor, “ANOVA-Like Differential Expression (ALDEx) Analysis for Mixed Population RNA-Seq,” *PLOS ONE*, vol. 8, no. 7, p. e67019, Jul. 2013, doi: 10.1371/journal.pone.0067019.


In [16]:
!Rscript run_aldex.R

aldex.clr: generating Monte-Carlo instances and clr values
operating in serial mode
removed rows with sums equal to zero
computing center with all features
data format is OK
dirichlet samples complete
transformation complete
aldex.ttest: doing t-test
running tests for each MC instance:
|------------(25%)----------(50%)----------(75%)----------|
aldex.effect: calculating effect sizes
operating in serial mode
sanity check complete
rab.all  complete
rab.win  complete
rab of samples complete
within sample difference calculated
between group difference calculated
group summaries calculated
effect size calculated
summarizing output
[1] "ALDEx2 results written to output/TCGA_LUSC_aldex_results.tsv"


In [17]:
assert os.path.exists("output/TCGA_LUSC_aldex_results.tsv")

## 4. Converting feature table from tsv to biom

We want to convert the original feature table from a tab-separated file (TSV) to the BIOM format. The BIOM format is a widely used file format for storing and representing biological observations. It is especially good at handling sparsity - or the fact that there may often be a lot of 0s in your data.

[3] D. McDonald et al., “The Biological Observation Matrix (BIOM) format or: how I learned to stop worrying and love the ome-ome,” *GigaScience*, vol. 1, no. 1, p. 7, Jul. 2012, doi: 10.1186/2047-217X-1-7.

In [18]:
!biom convert \
    -i output/TCGA_LUSC_expression_feature_table_filt.tsv \
    -o output/TCGA_LUSC_expression_feature_table_filt.biom \
    --table-type="OTU table" \
    --to-hdf5

## 5. Mapping Ensembl gene identifiers to HGNC

We might want to know which Ensembl features map to which HUGO features. First, we'll download the `GRCh38` gtf file.

In [19]:
!wget -P input/ ftp://ftp.ensembl.org/pub/release-99/gtf/homo_sapiens/Homo_sapiens.GRCh38.99.gtf.gz

--2020-02-28 12:02:41--  ftp://ftp.ensembl.org/pub/release-99/gtf/homo_sapiens/Homo_sapiens.GRCh38.99.gtf.gz
           => ‘input/Homo_sapiens.GRCh38.99.gtf.gz’
Resolving ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.8
Connecting to ftp.ensembl.org (ftp.ensembl.org)|193.62.193.8|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/release-99/gtf/homo_sapiens ... done.
==> SIZE Homo_sapiens.GRCh38.99.gtf.gz ... 46905912
==> PASV ... done.    ==> RETR Homo_sapiens.GRCh38.99.gtf.gz ... done.
Length: 46905912 (45M) (unauthoritative)

Homo_sapiens.GRCh38 100%[===================>]  44.73M  1.60MB/s    in 33s     

2020-02-28 12:03:17 (1.36 MB/s) - ‘input/Homo_sapiens.GRCh38.99.gtf.gz’ saved [46905912]



Next, we'll create a feature metadata file mapping the Ensembl ID to the gene name. The following series of bash commands converts the cumbersome-to-use `gtf` file to an easily readable TSV.

This `gtf` file looks like this:

`1    havana    gene    11869    14409    .    +    .    gene_id "ENSG00000223972"; gene_version "5    "; gene_name "DDX11L1"; gene_source "havana"; gene_biotype "transcribed_unprocessed_pseudo    gene";`

NOTE: This tutorial was written on macOS Mojave. Depending on your OS, some of these commands may differ slightly.

In [20]:
%%bash

echo -e "feature-id\tGene Name" > output/gene_matching.tsv

zgrep "havana\tgene" input/Homo_sapiens.GRCh38.99.gtf.gz | \
awk -F "\t" '{print $9}' | \
awk -F ";" -v OFS="\t" '{print $1, $3}' | \
sed -e "s/gene_[a-z]* //g" | \
tr -d \" >> output/gene_matching.tsv

head output/gene_matching.tsv

feature-id	Gene Name
ENSG00000223972	 DDX11L1
ENSG00000227232	 WASH7P
ENSG00000243485	 MIR1302-2HG
ENSG00000237613	 FAM138A
ENSG00000268020	 OR4G4P
ENSG00000240361	 OR4G11P
ENSG00000186092	 OR4F5
ENSG00000238009	 AL627309.1
ENSG00000239945	 AL627309.3


## 6. Running Qurro

Finally, we'll run Qurro.

In [21]:
!qurro \
    -r output/TCGA_LUSC_aldex_results.tsv \
    -t output/TCGA_LUSC_expression_feature_table_filt.biom \
    -sm output/sample_metadata.tsv \
    -fm output/gene_matching.tsv \
    -o output/qurro

/miniconda3/envs/qurro/lib/python3.6/site-packages/qurro/_df_utils.py:126: FutureWarning: SparseDataFrame is deprecated and will be removed in a future version.
Use a regular DataFrame whose columns are SparseArrays instead.

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  table_sdf = pd.SparseDataFrame(table.matrix_data, default_fill_value=0.0)
/miniconda3/envs/qurro/lib/python3.6/site-packages/pandas/core/sparse/frame.py:257: FutureWarning: SparseSeries is deprecated and will be removed in a future version.
Use a Series with sparse values instead.

    >>> series = pd.Series(pd.SparseArray(...))

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  sparse_index=BlockIndex(N, blocs, blens),
/miniconda3/envs/qurro/lib/python3.6/site-packages/pandas/core/frame.py:3471: FutureWarning: SparseSeries is deprecated and will be removed in a future version.
Use a Series with sparse values instead.

    >>> s

In [22]:
assert os.path.exists("output/qurro/index.html")

Open the `output/qurro/index.html` page in the web browser of your choice. From here you can explore all the options Qurro has to offer! As an example, we're going to navigate to the "Differential" selection menu and select `rab:win:Primary Tumor`. This column contains the median clr values for the `Primary Tumor` group of samples. You may also want to check the box that says `Fit bar widths to a constant plot width?` to more easily visualize the rank plot.

Next, select the top and bottom 5% of samples in the `Autoselecting Features` section. Click apply and you should see the rank plot on the top left highlight the relevant feature rankings.

Finally, change the `x-axis` field to `Sample Type` and check the box that says `Use boxplots for categorical data?`. You should see a boxplot appear that shows a clear separation between Primary Tumor and Solid Tissue Normal!

![](imgs/aldex_screenshot_annotated.png)

That's it for this tutorial but we encourage you to try out some of the other features in Qurro! For example, try comparing a different field in the x-axis or searching for features by gene name.

Please feel free to contact us or open an issue if you have a question or suggestion about using Qurro.